In [ ]:
import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("/content/all_upwork_jobs.xlsx.csv")

In [ ]:
df

,title,link,published_date,is_hourly,hourly_low,hourly_high,budget,country
0,Experienced Media Buyer For Solar Pannel and R...,https://www.upwork.com/jobs/Experienced-Media-...,2024-02-17 09:09:54+00:00,False,NaN,NaN,500.0,NaN
1,Full Stack Developer,https://www.upwork.com/jobs/Full-Stack-Develop...,2024-02-17 09:09:17+00:00,False,NaN,NaN,1100.0,United States
2,SMMA Bubble App,https://www.upwork.com/jobs/SMMA-Bubble-App_%7...,2024-02-17 09:08:46+00:00,True,10.0,30.0,NaN,United States
3,Talent Hunter Specialized in Marketing,https://www.upwork.com/jobs/Talent-Hunter-Spec...,2024-02-17 09:08:08+00:00,True,NaN,NaN,NaN,United States
4,Data Engineer,https://www.upwork.com/jobs/Data-Engineer_%7E0...,2024-02-17 09:07:42+00:00,False,NaN,NaN,650.0,India
...,...,...,...,...,...,...,...,...
213270,Looking for a React Developer,https://www.upwork.com/jobs/Looking-for-React-...,2024-03-14 22:22:00+00:00,True,15.0,35.0,NaN,Canada
213271,AI developer,https://www.upwork.com/jobs/developer_%7E010f3...,2024-03-18 04:24:11+00:00,True,5.0,20.0,NaN,United States
213272,Looking for Spanish and Portuguese recording p...,https://www.upwork.com/jobs/Looking-for-Spanis...,2024-03-15 06:35:50+00:00,False,NaN,NaN,500.0,China
213273,Website Page Speed Optimization,https://www.upwork.com/jobs/Website-Page-Speed...,2024-03-15 06:36:47+00:00,True,NaN,NaN,NaN,Australia


In [ ]:
# to show all rows
pd.set_option('display.max_rows',None)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244828 entries, 0 to 244827
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   title           244827 non-null  object 
 1   link            244827 non-null  object 
 2   published_date  244828 non-null  object 
 3   is_hourly       244828 non-null  bool   
 4   hourly_low      102422 non-null  float64
 5   hourly_high     98775 non-null   float64
 6   budget          103891 non-null  float64
 7   country         239751 non-null  object 
dtypes: bool(1), float64(3), object(4)
memory usage: 13.3+ MB


In [ ]:
# convert published_data to datetime formate
df['published_date']=pd.to_datetime(df['published_date'],errors='coerce')

In [ ]:
# fill misssing values

df['country'].fillna('Unknown',inplace=True)
df['hourly_low'].fillna(0,inplace=True)
df['hourly_high'].fillna(0,inplace=True)
df['budget'].fillna(0,inplace=True)


In [ ]:
# Remove rows with missing titles or links
df.dropna(subset=['title','link'],inplace=True)

In [ ]:
from os import times
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
from tqdm import tqdm  # Import tqdm for progress bar

def scrape_description(url):
    headers = {"User-Agent":"Mozilla/5.0"}

    try:

        response = requests.get(url,headers = headers,timeout=10)

        if response.status_code == 200:

            soup = BeautifulSoup(response.text,"html.parser")

            job_desc=soup.find("div",class_ ="job-description")

            return job_desc.text if job_desc else "Description not found" # Modified return statement

        else:

            return"Failed to retrive"

    except Exception as e:
        return str(e)

batch_size = 200  # Adjust as needed
total_links = len(df["link"])

for i in tqdm(range(0, total_links, batch_size), desc="Scraping Progress"):
    batch_links = df["link"][i:i + batch_size]
    df.loc[batch_links.index, "job_description"] = batch_links.apply(
        lambda x: scrape_description(x) if pd.notnull(x) else "No link"
    )

    # Save intermediate results (optional)
    df.to_csv("intermediate_results.csv", index=False)

Scraping Progress:  59%|█████▊    | 626/1067 [2:43:13<2:00:54, 16.45s/it]

In [ ]:
from google.colab import files # If you are using Google Colab
files.download('intermediate_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#import pandas as pd
#from tqdm import tqdm

# ... (your scrape_description function) ...

# 1. Load intermediate results
df_intermediate = pd.read_csv("intermediate_results.csv")

df_remaining = df_intermediate[df_intermediate["job_description"].isin(["No link", "Failed to retrieve", "Other error message"])]  # Replace "Other error message" if needed

# 3. Rescrape remaining data (now using the defined df_remaining)
for index, row in tqdm(df_remaining.iterrows(), total=len(df_remaining), desc="Resuming Scraping"):
    df_remaining.loc[index, "job_description"] = scrape_description(row["link"])
    if index % 100 == 0:
        df_remaining.to_csv("updated_intermediate_results.csv", index=False)

# 4. Merge with original data
df_final = pd.concat([df_intermediate[~df_intermediate.index.isin(df_remaining.index)], df_remaining])

# 5. Save final results
df_final.to_csv("final_results.csv", index=False)

In [ ]:
from google.colab import files #
files.download('final_results.csv')

# What is tqdm?

tqdm is a popular Python library that provides fast, extensible progress bars for loops and iterables.
It's incredibly useful for tracking the progress of time-consuming tasks, such as web scraping, data processing, or large file operations.

# What does from tqdm import tqdm do?

This line imports the tqdm function (or class) from the tqdm module.

from ... import ...: This is a standard Python syntax for importing specific parts of a module.

tqdm (module): Refers to the overall library or module that contains the progress bar functionality.
tqdm (function/class): This is the main function (or class) you'll use to create progress bars.

